# Exploring Cospectral Graphs with SMOL

Two graphs are **cospectral** if they share the same eigenvalues for a given matrix. This notebook explores cospectral pairs in the SMOL database and shows how different matrix types distinguish graphs.

SMOL provides spectra for 7 matrix types:
- Adjacency (A)
- Kirchhoff Laplacian (L = D - A)
- Signless Laplacian (Q = D + A)
- Normalized Laplacian (I - D^{-1/2} A D^{-1/2})
- Non-backtracking / Hashimoto (B)
- Non-backtracking Laplacian (I - D^{-1} B)
- Distance matrix (D[i,j] = shortest path length, connected graphs only)

In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import requests
from urllib.parse import quote

np.set_printoptions(precision=4, suppress=True)

# SMOL API endpoint
BASE_URL = "https://smol-graphs-db.fly.dev"

## Querying SMOL for Cospectral Pairs

Let's find adjacency-cospectral pairs in the SMOL database.

In [ ]:
# Get an adjacency-cospectral pair from SMOL
response = requests.get(f"{BASE_URL}/cospectral-pairs", params={"matrix": "adj", "n": 8, "limit": 1})
pairs = response.json()

if pairs:
    pair = pairs[0]
    g1_graph6 = pair['graph1']['graph6']
    g2_graph6 = pair['graph2']['graph6']
    
    print(f"Found adjacency-cospectral pair:")
    print(f"  Graph 1: {g1_graph6}")
    print(f"  Graph 2: {g2_graph6}")
    
    # Get full details for both graphs
    g1_detail = requests.get(f"{BASE_URL}/graph/{quote(g1_graph6, safe='')}").json()
    g2_detail = requests.get(f"{BASE_URL}/graph/{quote(g2_graph6, safe='')}").json()
    
    print(f"\nAdjacency eigenvalues (same for both):")
    print(f"  {g1_detail['spectra']['adj_eigenvalues']}")
    
    print(f"\nGraph 1: {g1_detail['m']} edges, tags: {g1_detail.get('tags', [])}")
    print(f"Graph 2: {g2_detail['m']} edges, tags: {g2_detail.get('tags', [])}")
    
    # Check other matrix types
    print(f"\nCospectral for other matrix types?")
    for matrix in ["kirchhoff", "signless", "lap", "nb", "nbl", "dist"]:
        mates = g1_detail.get('cospectral_mates', {}).get(matrix, [])
        is_cospec = g2_graph6 in mates
        print(f"  {matrix:10s}: {'Yes' if is_cospec else 'No'}")

In [ ]:
# Visualize the cospectral pair
G1 = nx.from_graph6_bytes(g1_graph6.encode())
G2 = nx.from_graph6_bytes(g2_graph6.encode())

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

nx.draw(G1, ax=axes[0], with_labels=True, node_color='lightblue', node_size=500)
axes[0].set_title(f'Graph 1: {g1_graph6}')

nx.draw(G2, ax=axes[1], with_labels=True, node_color='lightcoral', node_size=500)
axes[1].set_title(f'Graph 2: {g2_graph6}')

plt.suptitle('Adjacency-Cospectral Pair from SMOL', fontsize=14)
plt.tight_layout()
plt.show()

## Switching Mechanisms

SMOL includes detected switching mechanisms (like GM switching) that explain why graphs are cospectral.

In [ ]:
# Check for switching mechanisms
encoded = quote(g1_graph6, safe="")
mech_response = requests.get(f"{BASE_URL}/api/graph/{encoded}/mechanisms")

if mech_response.status_code == 200:
    data = mech_response.json()
    mechanisms = data.get('mechanisms', {})
    total_mechs = sum(len(mechs) for mechs in mechanisms.values())
    print(f"Switching mechanisms found: {total_mechs}")
    
    for matrix_type, mechs in mechanisms.items():
        for mech in mechs:
            if mech['mate'] == g2_graph6:
                print(f"\nMechanism for this pair:")
                print(f"  Type: {mech['mechanism']}")
                print(f"  Matrix: {matrix_type}")
                if mech.get('config'):
                    print(f"  Configuration: {mech['config']}")
else:
    print("No switching mechanisms found for this graph")

## Non-Backtracking Cospectrality

The non-backtracking (Hashimoto) matrix often distinguishes graphs that other matrices cannot. Let's find NB-cospectral pairs.

In [ ]:
# Get an NB-cospectral pair from SMOL
response = requests.get(f"{BASE_URL}/cospectral-pairs", params={"matrix": "nb", "n": 8, "limit": 1})
pairs = response.json()

if pairs:
    pair = pairs[0]
    nb_g1 = pair['graph1']['graph6']
    nb_g2 = pair['graph2']['graph6']
    
    print(f"Found NB-cospectral pair:")
    print(f"  Graph 1: {nb_g1}")
    print(f"  Graph 2: {nb_g2}")
    
    # Get full details
    detail = requests.get(f"{BASE_URL}/graph/{quote(nb_g1, safe='')}").json()
    
    nb_re = detail['spectra']['nb_eigenvalues_re']
    nb_im = detail['spectra']['nb_eigenvalues_im']
    print(f"\nNB eigenvalues (real part, first 5): {nb_re[:5]}")
    print(f"NB eigenvalues (imag part, first 5): {nb_im[:5]}")
    
    # Check if they're also cospectral for other matrices
    print(f"\nAlso cospectral for:")
    for matrix in ["adj", "kirchhoff", "signless", "lap"]:
        mates = detail.get('cospectral_mates', {}).get(matrix, [])
        if nb_g2 in mates:
            print(f"  - {matrix}")

## Database Statistics

SMOL contains ~12.3M graphs. Let's explore cospectrality statistics.

In [ ]:
# Get database statistics
stats_response = requests.get(f"{BASE_URL}/api/stats/mechanisms")
if stats_response.status_code == 200:
    mech_stats = stats_response.json()
    
    print("Cospectral pairs by matrix type (n=8 and n=9):")
    for matrix in ["adj", "kirchhoff", "signless", "lap", "nb", "nbl", "dist"]:
        count = mech_stats.get(f'{matrix}_pairs', 0)
        print(f"  {matrix:10s}: {count:,} pairs")
    
    print(f"\nSwitching mechanisms detected:")
    print(f"  GM switching (adj, n=8): {mech_stats.get('gm_adj_n8', 0):,}")
    print(f"  GM switching (adj, n=9): {mech_stats.get('gm_adj_n9', 0):,}")

# Get overall stats
stats_response = requests.get(f"{BASE_URL}/stats")
if stats_response.status_code == 200:
    stats = stats_response.json()
    print(f"\nTotal graphs in SMOL: {stats.get('total_graphs', 0):,}")
    print(f"Connected graphs: {stats.get('connected_graphs', 0):,}")